In [1]:
import numpy as np
import pickle
from scipy.spatial.transform import Rotation as R
import random
import torch
from tqdm.auto import tqdm
import open3d as o3d
import os.path
from collections import deque

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
robot_type = 'UR5'
tgt_pointcloud = np.loadtxt(f'../../Demo/models/{robot_type}_base_m_x5.txt')[:,0:3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(tgt_pointcloud)
pcd.estimate_normals()
# o3d.visualization.draw_geometries([pcd], point_show_normal=True)
pcd.orient_normals_consistent_tangent_plane(100)
#new_n = []
#for n in pcd.normals:
#    new_n.append(-1* n)
#pcd.normals = o3d.utility.Vector3dVector(np.asarray(new_n))
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
#         pcd, depth=9)
# print(mesh)
# o3d.visualization.draw_geometries([mesh],point_show_normal=True)

# radii = [0.005, 0.01, 0.02, 0.04]
# rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd, o3d.utility.DoubleVector(radii))
# o3d.visualization.draw_geometries([pcd, rec_mesh], point_show_normal=True)

In [3]:
# print (np.asarray(rec_mesh.vertices).shape)
# print (np.asarray(rec_mesh.triangles).shape)
print (np.asarray(pcd.normals).shape)

# iter_mesh = np.asarray(rec_mesh.triangles)

(33775, 3)


In [4]:
# point on sphere generation
scale = 5.0
D = 0.5 * scale # m
points_on_sphere = []
for alpha in range(10, 90, 10):
    z = D * np.sin(np.deg2rad(alpha) )
    D_ = D * np.cos(np.deg2rad(alpha) )
    
    for beta in range(0, 360, 20):
        x = D_ * np.cos(np.deg2rad(beta))
        y = D_ * np.sin(np.deg2rad(beta))
        if np.sqrt(x**2 + y**2) > 0.3 * scale:
            points_on_sphere.append([x,y,z])
points_on_sphere.reverse()
points_on_sphere = np.asarray(points_on_sphere)
print (len(points_on_sphere))
np.savetxt(f'{robot_type}_point_on_sphere.txt', points_on_sphere, fmt = '%1.6f')

90


In [5]:
for v_index, view_point in tqdm(enumerate(points_on_sphere), total = len(points_on_sphere)):
    visible_points = []
    for index, normal in enumerate(pcd.normals):
        rad = angle_between(normal, pcd.points[index] - view_point)
        angle = np.rad2deg(rad)
        if angle > 110:
            visible_points.append(pcd.points[index])
    visible_points.append(view_point)
    np.savetxt(f'./{robot_type}_visible_points/visible_points_{v_index:06d}.txt', np.asarray(visible_points))

  0%|          | 0/90 [00:00<?, ?it/s]